# CNN Network

In [24]:
import torch
import torch.nn.functional as F
import os
import numpy as np
from typing import Tuple
import torchvision
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models

print(torch.__version__)
print(torchvision.__version__)

curPath=os.path.abspath('')
dataPath=curPath+"/../data"
BATCH_SIZE=512
EPOCHS=1 
# DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu") 
DEVICE = torch.device("mps") 
print(DEVICE)
print(dataPath)


1.13.1
0.13.1a0
mps
/Users/jiaruiye/Desktop/FDU/专业课程/必修课程/人工智能/Projects/PJ1/Part3/../data


In [68]:
transform = transforms.Compose([
    transforms.ToTensor(),
    # transforms.Normalize((0.5,), (0.5,)),
    transforms.Normalize([0.1307,], [0.3081,]),
    transforms.Lambda(lambda x: x.repeat(3, 1, 1)),
    transforms.Resize((224, 224)),
    transforms.RandomRotation(15),
])

train_set = datasets.MNIST(root=dataPath, train=True, download=False, transform=transform)
test_set = datasets.MNIST(root=dataPath, train=False, download=False, transform=transform)
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=True)

model = models.resnet18(pretrained=True)
for param in model.parameters():
    param.requires_grad = False

# model.conv1 = torch.nn.Conv2d(1, 64, kernel_size=7, stride= 2, padding=3, bias= False)
model.fc = torch.nn.Linear(512, 10)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam([{'params':model.fc.parameters()}], lr=0.001)
model=model.to(DEVICE)
print(model)
print(len(train_loader))

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [26]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = F.nll_loss(output, target, reduction='sum')  # calculate loss
            test_loss += loss.item()  # add loss to running total
            pred = output.argmax(dim=1, keepdim=True)  # get predicted class
            correct += pred.eq(target.view_as(pred)).sum().item()  # count correct predictions

    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)
    print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
        test_loss, correct, len(test_loader.dataset), accuracy))
    return test_loss, accuracy
def train(model,device,train_loader,epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(data)
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()
        if batch_idx%30==0 or batch_idx==117:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(data), len(train_loader.dataset),
                    100. * batch_idx / len(train_loader), loss.item()))


In [69]:
# load the saved model
model.load_state_dict(torch.load('fineTuningCharacterClassificaion.pt'))

<All keys matched successfully>

In [70]:
for epoch in range(EPOCHS):
    train(model, DEVICE, train_loader, epoch)
    test(model, DEVICE, test_loader)

Train Epoch: 0 [0/60000 (0%)]	Loss: 0.236212
Train Epoch: 0 [15360/60000 (25%)]	Loss: 0.158955
Train Epoch: 0 [30720/60000 (51%)]	Loss: 0.195552
Train Epoch: 0 [46080/60000 (76%)]	Loss: 0.199715
Train Epoch: 0 [11232/60000 (99%)]	Loss: 0.192953
Test set: Average loss: -4.4764, Accuracy: 9466/10000 (95%)


In [63]:
# save the model
torch.save(model.state_dict(), 'fineTuningCharacterClassificaion.pt')